In [ ]:
%%bash

# create the singularity image on Argon

SINGULARITY_TMPDIR=/Users/tientong/singularity_temp
export SINGULARITY_TMPDIR
singularity build /Users/tientong/singularity_home/fmriprep_20.1.3.simg docker://poldracklab/fmriprep:20.1.3

# Create fMRIPrep template files

## Template for subject with fieldmap data

```
#!/bin/sh
#$ -pe smp 20
#$ -q UI,CCOM,all.q
#$ -ckpt user
#$ -o /Shared/tientong_scratch/abcd/code/derivative_logs/fmriprep
#$ -e /Shared/tientong_scratch/abcd/code/derivative_logs/fmriprep
OMP_NUM_THREADS=16
singularity run -H /Users/tientong/singularity_home \
  /Users/tientong/fmriprep_20.1.3.simg \
  /Shared/tientong_scratch/abcd/rawdata \
  /Shared/tientong_scratch/abcd/derivatives/fmriprep \
  participant --participant-label SUBJECT \
  --fs-license-file /Shared/oleary/functional/FreeSurferLicense/license.txt \
  -w /Shared/tientong_scratch/abcd/derivatives/fmriprep --write-graph \
  --output-spaces T1w MNI152NLin6Asym:res-2 --stop-on-first-crash \
  --omp-nthreads 16 --nthreads 16 --mem 40 --notrack --fs-no-reconall \
  --skip_bids_validation --ignore slicetiming
```

## Template for subject without fieldmap data

```
#!/bin/sh
#$ -pe smp 20
#$ -q UI,CCOM,all.q
#$ -ckpt user
#$ -o /Shared/tientong_scratch/abcd/code/derivative_logs/fmriprep
#$ -e /Shared/tientong_scratch/abcd/code/derivative_logs/fmriprep
OMP_NUM_THREADS=16
singularity run -H /Users/tientong/singularity_home \
  /Users/tientong/fmriprep_20.1.3.simg \
  /Shared/tientong_scratch/abcd/rawdata \
  /Shared/tientong_scratch/abcd/derivatives/fmriprep \
  participant --participant-label SUBJECT \
  --fs-license-file /Shared/oleary/functional/FreeSurferLicense/license.txt \
  -w /Shared/tientong_scratch/abcd/derivatives/fmriprep --write-graph \
  --output-spaces T1w MNI152NLin6Asym:res-2 --stop-on-first-crash \
  --omp-nthreads 16 --nthreads 16 --mem 40 --notrack --fs-no-reconall \
  --skip_bids_validation --ignore slicetiming fieldmaps --use-syn-sdc
```

## From those templates, create subject-specific fMRIPrep job script


In [ ]:
%%bash

list=($(cat /Shared/tientong_scratch/abcd/derivatives/baw/randomhash_200725.txt))
bidsdir=/Shared/tientong_scratch/abcd/rawdata
template=/Shared/tientong_scratch/abcd/code/fmriprep/template
script=/Shared/tientong_scratch/abcd/code/fmriprep/subject_scripts

for i in ${!list[@]} ; do
 SUBJECT=(`echo ${list[$i]} | awk '{gsub("/"," "); print $5}'| awk '{gsub("-"," "); print $2}'`)
 SESSION=baselineYear1Arm1
 if [ -d ${bidsdir}/sub-${SUBJECT}/ses-${SESSION}/func ] &&
    [ 0 -lt `ls ${bidsdir}/sub-${SUBJECT}/ses-${SESSION}/anat/*T1w*.nii.gz 2>/dev/null | wc -l` ] ; then
  STR=$'"IntendedFor"\: \[$'
  if [ -d ${bidsdir}/sub-${SUBJECT}/ses-${SESSION}/fmap ] &&
     [ 0 -lt `grep -rnw ${bidsdir}/sub-${SUBJECT}/ses-${SESSION}/fmap -e "$STR" 2>/dev/null | wc -l` ] ; then
    sed -e "s|SUBJECT|${SUBJECT}|g" ${template}/fmriprep_nostc_TEMPLATE.sh \
    > ${script}/sub-${SUBJECT}_ses-${SESSION}_fmriprep_nostc.sh
    # qsub ${script}/sub-${SUBJECT}_ses-${SESSION}_fmriprep_nostc.sh
  else
    sed -e "s|SUBJECT|${SUBJECT}|g" ${template}/fmriprep_nostc_syn_TEMPLATE.sh \
    > ${script}/sub-${SUBJECT}_ses-${SESSION}_fmriprep_nostc_syn.sh
    # qsub ${script}/sub-${SUBJECT}_ses-${SESSION}_fmriprep_nostc_syn.sh        
  fi
 fi
done


##### 838 jobs - submitting 100 at once

jobs=($(ls /Shared/tientong_scratch/abcd/code/fmriprep/subject_scripts/*))

for i in $(seq 0 1 100) ; do
  qsub ${jobs[$i]}
done


### delete intermediate files of subjects that completed fMRIPrep

In [ ]:
%%bash
# First delete the intermediate files for subjects with completed fmriprep

template=/Shared/tientong_scratch/abcd/code/fmriprep/template
fmriprepdir=/Shared/tientong_scratch/abcd/derivatives/fmriprep
script=/Shared/tientong_scratch/abcd/code/fmriprep/subject_scripts
logdir=/Shared/tientong_scratch/abcd/code/derivative_logs/fmriprep

source /etc/profile.d/sge.sh

cd $fmriprepdir/fmriprep
finish=($(ls *.html))
SUBJECT_finish=()
for i in ${!finish[@]} ; do
  SUBJECT_finish+=($(cut -d "." -f 1 <<<`echo ${finish[$i]}  | awk '{gsub("-"," "); print $2}'`)) 
done
cd
for i in ${!SUBJECT_finish[@]} ; do
  sed -e "s|SUBJECT|${SUBJECT_finish[$i]}|g" ${template}/delete.sh \
       > ${template}/sub-${SUBJECT_finish[$i]}_delete.sh
  qsub ${template}/sub-${SUBJECT_finish[$i]}_delete.sh 
done

save this file as `/Users/tientong/cron_delete.sh` and run the shell below on Argon to run the script every hour using `crontab -e`

```
0 * * * * /Users/tientong/cron_delete.sh
```

Check to see if this is saved with `crontab -l`, and delete with `crontab -r`

# fMRIPrep QC: Check for errors

Jobs submitted to all.q will be kicked off but SHOULD be automatically resubmitted. However, in case there are errors in this process, find the subjects with errors, delete the fmriprep outputs for those subjects, and rerun

In [ ]:
%%bash

template=/Shared/tientong_scratch/abcd/code/fmriprep/template
fmriprepdir=/Shared/tientong_scratch/abcd/derivatives/fmriprep
script=/Shared/tientong_scratch/abcd/code/fmriprep/subject_scripts
logdir=/Shared/tientong_scratch/abcd/code/derivative_logs/fmriprep


# Get an array of subjects who completed fmriprep

cd $fmriprepdir/fmriprep
finish=($(ls *.html))
SUBJECT_finish=()
for i in ${!finish[@]} ; do
  SUBJECT_finish+=($(cut -d "." -f 1 <<<`echo ${finish[$i]}  | \
                     awk '{gsub("-"," "); print $2}'`)) 
done
cd

# get an array of subjects need to be run
subjectlist=($(cat /Shared/tientong_scratch/abcd/derivatives/baw/randomhash_200725.txt))
SUBJECT_full=()
for i in ${!subjectlist[@]} ; do
  SUBJECT_full+=($(echo ${subjectlist[$i]} | \
                   awk '{gsub("/"," "); print $5}'| \
                   awk '{gsub("-"," "); print $2}'))
done

# get the difference between the completed list and to-run list
SUBJECT=($(echo ${SUBJECT_full[@]} ${SUBJECT_finish[@]} | \
           tr ' ' '\n' | sort | uniq -u))

for i in ${!SUBJECT[@]} ; do
  if [ -f $script/*${SUBJECT[$i]}* ] ; then
     rm $logdir/*${SUBJECT[$i]}*
     rm -rf $fmriprepdir/fmriprep/*${SUBJECT[$i]}*/*
     rmdir $fmriprepdir/fmriprep/*${SUBJECT[$i]}*
     rm -rf $fmriprepdir/fmriprep_wf/*${SUBJECT[$i]}*
     qsub $script/*${SUBJECT[$i]}*
  fi
done


In [ ]:
%%bash

fmriprepdir=/Shared/tientong_scratch/abcd/derivatives/fmriprep
script=/Shared/tientong_scratch/abcd/code/fmriprep/subject_scripts
logdir=/Shared/tientong_scratch/abcd/code/derivative_logs/fmriprep

cd $logdir
STR1=$'Traceback (most recent call last)'
STR2=$'Could not generate CITATION.'
log_w_err=($(grep -l "$STR1\|$STR2" `ls ${logdir}`))

grep -l "$STR1\|$STR2" `ls ${logdir}` | wc -l

for i in ${!log_w_err[@]} ; do
 SUBJECT=`echo ${log_w_err[$i]} | awk '{gsub("_"," "); print $1}' | 
           awk '{gsub("-"," "); print $2}'`
 rm -rf $fmriprepdir/fmriprep/*$SUBJECT*
 rm -rf $fmriprepdir/fmriprep_wf/*$SUBJECT*
 rm -rf $logdir/*$SUBJECT*
 #qsub $script/*$SUBJECT*
done

cd


for i in ${!log_w_err[@]} ; do
 SUBJECT=`echo ${log_w_err[$i]} | awk '{gsub("_"," "); print $1}' | 
           awk '{gsub("-"," "); print $2}'`
 rm -rf $fmriprepdir/fmriprep/*$SUBJECT*
 rm -rf $logdir/*$SUBJECT*
 sed -e "s|SUBJECT|${SUBJECT}|g" ${template}/delete.sh > ${template}/sub-${SUBJECT}_delete.sh
 qsub ${template}/sub-${SUBJECT}_delete.sh  
 #qsub $script/*$SUBJECT*
done

cd

In [ ]:
fMRIPrep finished successfully!

NDARINV73J1L7TG - only have 10s of data??

In [ ]:
grep -l "No errors to report" `ls fmriprep/*.html` | wc -l

grep -r code/derivative_logs/fmriprep \
     -e "fMRIPrep finished successfully" | wc -l

non_steady_state_outlier_XX - columns indicate non-steady state volumes with a single 1 value and 0 elsewhere (i.e., there is one non_steady_state_outlier_XX column per outlier/volume). Source code https://github.com/nipy/nipype/blob/929de3595/nipype/algorithms/confounds.py#L974-L995

ChrisGorgolewski

    Another is adding the artefact regressors produced by FMRIPREP to your GLM. This will essentially remove any variance associated with those scans which could otherwise lead to false results.


Additionally, a set of physiological regressors were extracted to allow for component-based noise correction (CompCor, Behzadi et al. 2007). Principal components are estimated after high-pass filtering the preprocessed BOLD time-series **(using a discrete cosine filter with 128s cut-off = 0.008 Hz)** for the two CompCor variants: temporal (tCompCor) and anatomical (aCompCor). tCompCor components are then calculated from the top 5% variable voxels within a mask covering the subcortical regions. This subcortical mask is obtained by heavily eroding the brain mask, which ensures it does not include cortical GM regions. For aCompCor, components are calculated within the intersection of the aforementioned mask and the union of CSF and WM masks calculated in T1w space, after their projection to the native space of each functional run (using the inverse BOLD-to-T1w transformation). Components are also calculated separately within the WM and CSF masks. For each CompCor decomposition, the k components with the largest singular values are retained, such that the retained components’ time series are sufficient to explain 50 percent of variance across the nuisance mask (CSF, WM, combined, or temporal). The remaining components are dropped from consideration.

fMRIPrep does high-pass filtering before running anatomical or temporal CompCor. Therefore, when using CompCor regressors, the corresponding cosine_XX regressors should also be included in the design matrix.


# Notes on Discrete cosine-basis regressors

Discrete cosine-basis regressors. Physiological and instrumental (scanner) noise sources are generally present in fMRI data, typically taking the form of low-frequency signal drifts. To account for these drifts, temporal high-pass filtering is the immediate option. Alternatively, low-frequency regressors can be included in the statistical model to account for these confounding signals. Using the DCT basis functions, fMRIPrep generates these low-frequency predictors:

cosine_XX - DCT-basis regressors.

One characteristic of the cosine regressors is that they are identical for two different datasets with the same TR and the same effective number of time points (effective length). It is relevant to mention effective because initial time points identified as nonsteady states are removed before generating the cosine regressors.


Chris Markiewicz:

    I mean the regressors are calculated purely from the repetition time (1/sampling rate) and the number of samples, and not at all from the actual values in your data files. Thus, if you have a 1.5s TR and 400 volumes for any two files, the cosine_xx regressors will be identical.

    You can also calculate them directly: https://github.com/nipy/nipype/blob/63e5ef25eeaf8717ef271cc3397b41c2e754c5ef/nipype/algorithms/confounds.py#L1489-L1522

    The reason we're producing cosine regressors is that we do high-pass filtering before running a/tCompCor, and these are all calculated on the series without non-steady-state volumes. So if you include CompCor regressors, you should also include cosine regressors.



Hagler et al. 2019 Neuroimage

A total of 16 initial frames (12.8 s) are discarded. On Siemens and Philips scanners, the first eight frames make up the pre-scan reference, and are not saved as DICOMS. An additional eight frames are discarded as part of the pre-analysis processing, for a total of 16 initial frames. 

On GE scanners with soft-ware version DV25, the first 12 frames make up the pre-scan reference. Instead of being discarded, those 12 reference scans are combined into one, and saved as the first frame, for a total of five initial frames to be discarded as part of the pre-analysis processing for GE DV25 series. 

On GE scanners with software version DV26, the pre-scan reference is not retained at all, and a total of 16 initial frames are discarded for GE DV26 scans as part of the pre-analysis processing.

In [6]:
import pandas as pd
import glob
import os

In [25]:
fmriprep="/Shared/tientong_scratch/abcd/derivatives/fmriprep/fmriprep/"
sub="sub-NDARINVT345WWLE"
ses="ses-baselineYear1Arm1"

jsonlist=glob.glob(fmriprep + sub + "/" + ses +
                   "/func/*_desc-confounds_regressors.json")


test = pd.read_json(jsonlist[2]).T.sort_values(by=['Mask','SingularValue'],
                                       ascending=False)

test.to_csv(fmriprep + sub + "/" + ses + "/func/" + 'test.csv', 
            sep='\t', index = True, header = True)

In [ ]:
%%bash

# create a file of non-steady state counts

list=($(ls /Shared/tientong_scratch/abcd/derivatives/fmriprep/fmriprep/*.html))
for i in ${!list[@]} ; do
sub=$(cut -d "." -f 1 <<<`echo ${list[$i]} | awk '{gsub("/"," ") ; print $7}'`)
/Shared/tientong_scratch/abcd/code/postfMRIPrep_misc_count-nonsteady.R \
 --path_fmriprep /Shared/tientong_scratch/abcd/derivatives/fmriprep/fmriprep/ \
 --subject $sub \
 --visit ses-baselineYear1Arm1 \
 --output /Shared/tientong_scratch/abcd/derivatives/fmriprep/count_nonsteady.txt
done

# How to load pklz files

In [ ]:
from nipype.utils.filemanip import loadpkl

In [ ]:
file='/Shared/tientong_scratch/abcd/derivatives/fmriprep/fmriprep_wf/single_subject_NDARINV1M0VVR9Y_wf/func_preproc_ses_baselineYear1Arm1_task_rest_run_01_wf/bold_std_trans_wf/bold_reference_wf/enhance_and_skullstrip_bold_wf/_std_target_MNI152NLin2009cAsym./unifize/result_unifize.pklz'
res=loadpkl(file)
print(res.inputs)
print('\n')
print(res.interface)
print('\n')
print(res.outputs)
print('\n')
print(res.runtime)